In [ ]:
%load_ext autoreload
%autoreload 2

import os

CURRENT_DPATH = os.path.abspath(os.path.dirname("__file__"))
PROJECT_ROOT = os.path.abspath(os.path.join(CURRENT_DPATH, os.pardir))
DATA_DPATH = os.path.join(PROJECT_ROOT, "external_data")

import matplotlib.pyplot as plt

from lane_detection_hackathon.datasets import DatasetMode, FileDataset
from lane_detection_hackathon.utils.fs import read_image, VideoReader, VideoWriter
from lane_detection_hackathon.inference import SegmentationInference
from lane_detection_hackathon.baseparser import BaseParser
from lane_detection_hackathon.utils.image import overlay

In [ ]:
MODEL_ID = "40446a2b3b7543c292301a3b2da1ed67"
CHECKPOINT_DPATH = os.path.join(PROJECT_ROOT, "train_checkpoints")
MODEL_FNAME = f"best-valid-iou_{MODEL_ID}.pth"

MODEL_FPATH = os.path.join(CHECKPOINT_DPATH, MODEL_FNAME)

inference = SegmentationInference.from_file(
  MODEL_FPATH, device="cuda", batch_size=32, verbose=False
)

In [ ]:
video_dpath = os.path.join(DATA_DPATH, "test_videos")
video_dpath

In [ ]:
video_fname = "solidWhiteRight.mp4"
video_fname = "EV_zawrka_1_right_line.mp4"
# video_fname = "VID_20230104_111333.mp4"
video_fname = "Пулково_заход,_уход_на_второй,_посадка_online_video_cutter_com.mp4"

video_fpath = os.path.join(video_dpath, video_fname)

video_fpath

In [ ]:
output_dpath = os.path.join(DATA_DPATH, "output_videos")
os.makedirs(output_dpath, exist_ok=True)

output_fpath = os.path.join(output_dpath, f"output_{video_fname}")
output_fpath

In [ ]:
fps = 30 

with VideoWriter(output_fpath, fps=fps) as vider_writer:
    with VideoReader(video_fpath, verbose=True) as video_reader: 
        for frame in video_reader.get_frames():
            pred_result = inference.predict(frame)
            pred_mask = pred_result.get_rgb_mask(frame.shape[:2])
            overlayed_frame = overlay(frame, pred_mask)
            
            vider_writer.write_frame(overlayed_frame)
